In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = df['koi_disposition']

In [5]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855,4.578,0.797,284.50391,42.463860,15.770
2879,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328,4.481,0.963,295.50211,38.983540,13.099
107,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768,4.536,0.779,291.15878,40.750271,15.660
29,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712,4.359,1.082,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [9]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
from tensorflow.keras.utils import to_categorical
# One-hot encoding
#y_train_categorical = to_categorical(y_train)
#y_test_categorical = to_categorical(y_test)

# Train the Model



In [11]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=15))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                800       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 953
Trainable params: 953
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
5243/5243 - 0s - loss: 0.9406 - acc: 0.5276
Epoch 2/60
5243/5243 - 0s - loss: 0.8469 - acc: 0.5968
Epoch 3/60
5243/5243 - 0s - loss: 0.8094 - acc: 0.6208
Epoch 4/60
5243/5243 - 0s - loss: 0.7827 - acc: 0.6384
Epoch 5/60
5243/5243 - 0s - loss: 0.7619 - acc: 0.6584
Epoch 6/60
5243/5243 - 0s - loss: 0.7472 - acc: 0.6679
Epoch 7/60
5243/5243 - 0s - loss: 0.7351 - acc: 0.6780
Epoch 8/60
5243/5243 - 0s - loss: 0.7244 - acc: 0.6824
Epoch 9/60
5243/5243 - 0s - loss: 0.7154 - acc: 0.6847
Epoch 10/60
5243/5243 - 0s - loss: 0.7079 - acc: 0.6872
Epoch 11/60
5243/5243 - 0s - loss: 0.7018 - acc: 0.6922
Epoch 12/60
5243/5243 - 0s - loss: 0.6975 - acc: 0.6922
Epoch 13/60
5243/5243 - 0s - loss: 0.6933 - acc: 0.6908
Epoch 14/60
5243/5243 - 0s - loss: 0.6892 - acc: 0.6964
Epoch 15/60
5243/5243 - 0s - loss: 0.6844 - acc: 0.7015
Epoch 16/60
5243/5243 - 0s - loss: 0.6829 - acc: 0.7002
Epoch 17/60
5243/5243 - 0s - loss: 0.6783 - acc: 0.7021
Epoch 18/60
5243/5243 - 0s - loss: 0.6754 - acc: 0.7034
E

In [15]:
model_loss, model_accuracy = model.evaluate(
    X_train_scaled, y_train_categorical, verbose=2)
print(
    f"Train Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

5243/5243 - 0s - loss: 0.6207 - acc: 0.7252
Train Neural Network - Loss: 0.6207136548181629, Accuracy: 0.7251573801040649


In [16]:
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=15))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               1600      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 12,003
Trainable params: 12,003
Non-trainable params: 0
_________________________________________________________________


In [17]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_test_scaled,
    y_test_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
1748/1748 - 0s - loss: 0.9405 - acc: 0.5240
Epoch 2/100
1748/1748 - 0s - loss: 0.8281 - acc: 0.6093
Epoch 3/100
1748/1748 - 0s - loss: 0.7832 - acc: 0.6436
Epoch 4/100
1748/1748 - 0s - loss: 0.7568 - acc: 0.6545
Epoch 5/100
1748/1748 - 0s - loss: 0.7349 - acc: 0.6728
Epoch 6/100
1748/1748 - 0s - loss: 0.7173 - acc: 0.6842
Epoch 7/100
1748/1748 - 0s - loss: 0.7011 - acc: 0.6876
Epoch 8/100
1748/1748 - 0s - loss: 0.6885 - acc: 0.7002
Epoch 9/100
1748/1748 - 0s - loss: 0.6808 - acc: 0.7008
Epoch 10/100
1748/1748 - 0s - loss: 0.6655 - acc: 0.6991
Epoch 11/100
1748/1748 - 0s - loss: 0.6590 - acc: 0.7105
Epoch 12/100
1748/1748 - 0s - loss: 0.6527 - acc: 0.7168
Epoch 13/100
1748/1748 - 0s - loss: 0.6476 - acc: 0.7294
Epoch 14/100
1748/1748 - 0s - loss: 0.6398 - acc: 0.7208
Epoch 15/100
1748/1748 - 0s - loss: 0.6312 - acc: 0.7323
Epoch 16/100
1748/1748 - 0s - loss: 0.6201 - acc: 0.7346
Epoch 17/100
1748/1748 - 0s - loss: 0.6264 - acc: 0.7311
Epoch 18/100
1748/1748 - 0s - loss: 0.60

In [18]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Test Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
#print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

1748/1748 - 0s - loss: 0.2461 - acc: 0.9142
Test Neural Network - Loss: 0.24610876968850806, Accuracy: 0.9141876697540283


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [29]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=15))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, y_train_categorical)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
5243/5243 - 0s - loss: 0.5342 - acc: 0.7362
Epoch 2/50
5243/5243 - 0s - loss: 0.4754 - acc: 0.7671
Epoch 3/50
5243/5243 - 0s - loss: 0.4512 - acc: 0.7805
Epoch 4/50
5243/5243 - 0s - loss: 0.4348 - acc: 0.7902
Epoch 5/50
5243/5243 - 0s - loss: 0.4236 - acc: 0.7971
Epoch 6/50
5243/5243 - 0s - loss: 0.4148 - acc: 0.8010
Epoch 7/50
5243/5243 - 0s - loss: 0.4079 - acc: 0.8049
Epoch 8/50
5243/5243 - 0s - loss: 0.4039 - acc: 0.8056
Epoch 9/50
5243/5243 - 0s - loss: 0.4000 - acc: 0.8076
Epoch 10/50
5243/5243 - 0s - loss: 0.3954 - acc: 0.8119
Epoch 11/50
5243/5243 - 0s - loss: 0.3934 - acc: 0.8116
Epoch 12/50
5243/5243 - 0s - loss: 0.3893 - acc: 0.8124
Epoch 13/50
5243/5243 - 0s - loss: 0.3883 - acc: 0.8114
Epoch 14/50
5243/5243 - 0s - loss: 0.3879 - acc: 0.8144
Epoch 15/50
5243/5243 - 0s - loss: 0.3797 - acc: 0.8187
Epoch 16/50
5243/5243 - 0s - loss: 0.3799 - acc: 0.8182
Epoch 17/50
5243/5243 - 0s - loss: 0.3769 - acc: 0.8204
Epoch 18/50
5243/5243 - 0s - loss: 0.3742 - acc: 0.8231
E

In [30]:
print(grid.best_params_)
print(grid.best_score_)

{'batch_size': 100, 'epochs': 50}
0.797444218991709


In [31]:
#Re-run deep_model with hypertuned paramters for neural network model:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_test_scaled,
    y_test_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
1748/1748 - 0s - loss: 0.2688 - acc: 0.9005
Epoch 2/50
1748/1748 - 0s - loss: 0.2519 - acc: 0.9125
Epoch 3/50
1748/1748 - 0s - loss: 0.2486 - acc: 0.9193
Epoch 4/50
1748/1748 - 0s - loss: 0.2554 - acc: 0.9130
Epoch 5/50
1748/1748 - 0s - loss: 0.2468 - acc: 0.9125
Epoch 6/50
1748/1748 - 0s - loss: 0.2480 - acc: 0.9130
Epoch 7/50
1748/1748 - 0s - loss: 0.2391 - acc: 0.9199
Epoch 8/50
1748/1748 - 0s - loss: 0.2392 - acc: 0.9222
Epoch 9/50
1748/1748 - 0s - loss: 0.2296 - acc: 0.9216
Epoch 10/50
1748/1748 - 0s - loss: 0.2302 - acc: 0.9233
Epoch 11/50
1748/1748 - 0s - loss: 0.2303 - acc: 0.9205
Epoch 12/50
1748/1748 - 0s - loss: 0.2405 - acc: 0.9170
Epoch 13/50
1748/1748 - 0s - loss: 0.2286 - acc: 0.9262
Epoch 14/50
1748/1748 - 0s - loss: 0.2253 - acc: 0.9268
Epoch 15/50
1748/1748 - 0s - loss: 0.2218 - acc: 0.9291
Epoch 16/50
1748/1748 - 0s - loss: 0.2303 - acc: 0.9193
Epoch 17/50
1748/1748 - 0s - loss: 0.2323 - acc: 0.9199
Epoch 18/50
1748/1748 - 0s - loss: 0.2203 - acc: 0.9285
E

In [32]:
print("RESULTS COMPARISSON:")
print("First trained/test deep_model: Accuracy= 0.914  Loss = 0.246")
print("Same deep_model/data with With tuned parameters: Accuracy = 0.9548 Loss = 0.1566")

RESULTS COMPARISSON:
First trained/test deep_model: Accuracy= 0.914  Loss = 0.246
Same deep_model/data with With tuned parameters: Accuracy = 0.9548 Loss = 0.1566


# Save the Model

In [33]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'GCA_final.sav'
#joblib.dump(model1, filename)